In [1]:
import requests
import pandas as pd
from typing import Text
from flask import Flask, render_template
import pandas as pd
import requests
from datetime import timedelta, datetime, date
import time


In [76]:
url2 = 'https://fantasy.premierleague.com/api/event/32/live/'
liveInfo = requests.get(url2).json()['elements']
liveInfo = liveInfo[388-1]['explain']
playerInfo = []
for stats in liveInfo:
    for stat in stats['stats']:
        tempKeyList = (list(stat.keys())[0]))
        if (list(stat.keys())[0]))

SyntaxError: unmatched ')' (<ipython-input-76-f8756ac4d2b6>, line 8)

In [78]:
liveInfo.any()

[{'fixture': 286,
  'stats': [{'identifier': 'minutes', 'points': 0, 'value': 0}]},
 {'fixture': 312,
  'stats': [{'identifier': 'minutes', 'points': 2, 'value': 90},
   {'identifier': 'goals_scored', 'points': 8, 'value': 2},
   {'identifier': 'bonus', 'points': 2, 'value': 2}]}]